# Missing Data 

## When and Why data might go missing

By “missing” we simply mean NA (“not available”) or “not present for whatever reason”. Many data sets simply arrive with missing data, either because it exists and was not collected or it never existed. For example, in a collection of financial time series, some of the time series might start on different dates. Thus, values prior to the start date would generally be marked as missing.

In pandas, one of the most common ways that missing data is introduced into a data set is by reindexing. For example:

In [2]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.random.randn(5, 3), index=['a', 'c', 'e', 'f', 'h'],  columns=['one', 'two', 'three'])
   

In [3]:
df

,one,two,three
a,-1.210366,0.672701,1.897815
c,-1.539072,1.412019,-1.157953
e,-1.509576,-1.596040,-0.081748
f,-0.579853,-0.986653,-2.401565
h,0.647470,0.694876,0.287114


In [4]:
df['four'] = 'bar'

In [5]:
df['five'] = df['one'] > 0


In [6]:
df

,one,two,three,four,five
a,-1.210366,0.672701,1.897815,bar,False
c,-1.539072,1.412019,-1.157953,bar,False
e,-1.509576,-1.596040,-0.081748,bar,False
f,-0.579853,-0.986653,-2.401565,bar,False
h,0.647470,0.694876,0.287114,bar,True


In [7]:
df2 = df.reindex(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'])
df3=df2

In [8]:
df2

,one,two,three,four,five
a,-1.210366,0.672701,1.897815,bar,False
b,NaN,NaN,NaN,NaN,NaN
c,-1.539072,1.412019,-1.157953,bar,False
d,NaN,NaN,NaN,NaN,NaN
e,-1.509576,-1.596040,-0.081748,bar,False
f,-0.579853,-0.986653,-2.401565,bar,False
g,NaN,NaN,NaN,NaN,NaN
h,0.647470,0.694876,0.287114,bar,True



As data comes in many shapes and forms, pandas aims to be flexible with regard to handling missing data. While NaN is the default missing value marker for reasons of computational speed and convenience, we need to be able to easily detect this value with data of different types: floating point, integer, boolean, and general object. In many cases, however, the Python None will arise and we wish to also consider that “missing” or “not available” or “NA”.

Note If you want to consider inf and -inf to be “NA” in computations, you can set pandas.options.mode.use_inf_as_na = True.


## isna() and notna()

To make detecting missing values easier (and across different array dtypes), pandas provides the isna() and notna() functions, which are also methods on Series and DataFrame objects:

In [11]:
df2['one']

a   -1.210366
b         NaN
c   -1.539072
d         NaN
e   -1.509576
f   -0.579853
g         NaN
h    0.647470
Name: one, dtype: float64

In [10]:
pd.isna(df2['one'])

a    False
b     True
c    False
d     True
e    False
f    False
g     True
h    False
Name: one, dtype: bool

In [13]:
pd.notna(df2['one'])

a     True
b    False
c     True
d    False
e     True
f     True
g    False
h     True
Name: one, dtype: bool

## Inserting missing data

In [12]:
s = pd.Series([1, 2, 3])

In [15]:
s

0    1
1    2
2    3
dtype: int64

In [16]:
s.loc[0] = None

In [17]:
s

0    NaN
1    2.0
2    3.0
dtype: float64

## Calculations with missing data¶

When summing data, NA (missing) values will be treated as zero.
If the data are all NA, the result will be 0.
Cumulative methods like cumsum() and cumprod() ignore NA values by default, but preserve them in the resulting arrays. To override this behaviour and include NA values, use skipna=False.

In [20]:
df3

,one,two,three,five
a,-0.776022,-0.053394,0.759068,False
b,NaN,NaN,NaN,NaN
c,-0.445535,0.047644,-0.311413,False
d,NaN,NaN,NaN,NaN
e,0.093714,0.402714,-0.049590,True
f,-0.057853,-1.940590,-1.134919,False
g,NaN,NaN,NaN,NaN
h,-0.170220,0.187619,0.757137,False


In [13]:
sum1 = df3['one']+df3['two']

In [24]:
sum1

a   -0.829416
b         NaN
c   -0.397892
d         NaN
e    0.496428
f   -1.998443
g         NaN
h    0.017400
dtype: float64

In [25]:
sum1.mean()   ## NaN values are treated as zero

-0.5423844461223066

In [26]:
sum1.cumsum()

a   -0.829416
b         NaN
c   -1.227308
d         NaN
e   -0.730880
f   -2.729322
g         NaN
h   -2.711922
dtype: float64

In [28]:
sum1.cumsum(skipna=False)

a   -0.829416
b         NaN
c         NaN
d         NaN
e         NaN
f         NaN
g         NaN
h         NaN
dtype: float64

## Cleaning / filling missing data¶

## fillna() can “fill in” NA values with non-NA data in a couple of ways, which we illustrate:

Replace NA with a scalar value

In [29]:
sum1.fillna(0)

a   -0.829416
b    0.000000
c   -0.397892
d    0.000000
e    0.496428
f   -1.998443
g    0.000000
h    0.017400
dtype: float64

In [30]:
sum1.fillna('missing')

a    -0.829416
b      missing
c    -0.397892
d      missing
e     0.496428
f     -1.99844
g      missing
h    0.0173999
dtype: object

## Fill gaps forward or backward

Using the same filling arguments as reindexing, we can propagate non-NA values forward or backward:

In [32]:
sum1.fillna(method='pad')

a   -0.829416
b   -0.829416
c   -0.397892
d   -0.397892
e    0.496428
f   -1.998443
g   -1.998443
h    0.017400
dtype: float64

If we only want consecutive gaps filled up to a certain number of data points, we can use the limit keyword:

In [34]:
sum1

a   -0.829416
b         NaN
c   -0.397892
d         NaN
e    0.496428
f   -1.998443
g         NaN
h    0.017400
dtype: float64

##  Interpolation

Both Series and DataFrame objects have interpolate() that, by default, performs linear interpolation at missing datapoints.

In [40]:
sum1

a   -0.829416
b         NaN
c   -0.397892
d         NaN
e    0.496428
f   -1.998443
g         NaN
h    0.017400
dtype: float64

sum1.interpolate()
 
#In [63]: ts.interpolate().plot()

In [47]:
interpolated

8

## Replacing Generic Values

In [ ]:
Often times we want to replace arbitrary values with other values.

replace() in Series and replace() in DataFrame provides an efficient yet flexible way to perform such replacements.

For a Series, you can replace a single value or a list of values by another value:

In [49]:
ser = pd.Series([0., 1., 2., 3., 4.])

In [53]:
ser

0    0.0
1    1.0
2    2.0
3    3.0
4    4.0
dtype: float64

In [51]:
ser.replace(0, 5)

0    5.0
1    1.0
2    2.0
3    3.0
4    4.0
dtype: float64

In [15]:
d = {'a': list(range(4)), 'b': list('ab..'), 'c': ['a', 'b', np.nan, 'd']}

In [16]:
df = pd.DataFrame(d)

In [17]:
df

,a,b,c
0,0,a,a
1,1,b,b
2,2,.,NaN
3,3,.,d


In [56]:
df.replace('.', np.nan)

,a,b,c
0,0,a,a
1,1,b,b
2,2,NaN,NaN
3,3,NaN,d


In [57]:
df.replace(r'\s*\.\s*', np.nan, regex=True)  ## remove the white spaces

,a,b,c
0,0,a,a
1,1,b,b
2,2,NaN,NaN
3,3,NaN,d


In [58]:
 df.replace(['a', '.'], ['b', np.nan])

,a,b,c
0,0,b,b
1,1,b,b
2,2,NaN,NaN
3,3,NaN,d
